# 期货持仓报告

COT报告也称为期货持仓报告，由美国CFTC公布，包含美国期货市场上大型交易商的持仓明细，例如持有多少多头合约，空头合约和未平仓合约等。

COT报告每周公布一次，公布时间是美东时间周五下午，数据采集时间截止到公布当周周二。

#### 报告类型

1. Legacy - 传统报告
2. Supplemental - 补充报告
3. Disaggregated - 分类报告
4. Traders in financial futures - 金融期货交易者报告

传统报告，分类报告，金融期货交易者报告又可以细分为两种：仅包含期货，合并期货和期权。

#### 传统报告

传统报告将市场参与者分为两类：非商业交易者和商业交易者。

* 非商业(noncommercial): 一般指商品生产商或制造商，主要用期货对冲产品价格风险。
* 商业(commercial): 一般指投机性头寸，例如对冲基金或投行持仓的合约。

#### 补充报告

补充报告包含13种精选农产品合约，把市场参与者分为三类：非商业，商业和指数交易者。

#### 分类报告

分类报告是传统报告的深入，将市场参与者进一步细分：

* Producer/Merchant/Processor/User: 生产商/商家/加工/用户，一般用期货来对冲商品风险
* Swap Dealers: 掉期交易商，期货头寸主要用于对冲掉期交易的风险，对手方既可能是生产商，也可能是对冲基金
* Managed Money: 管理基金，指专门进行期货交易的实体，例如CTA, CPO, 或者对冲基金等
* Other Reportables: 其它报告，指小型交易者

#### 金融期货交易者报告

金融期货交易者报告包括金融合约，例如货币，美国国债，欧洲美元，股票和彭博商品指数等。该报告将市场参与者分为四类：

* Dealer/Intermediary: 做市商/中介，这些参与者被视为市场“卖方”，它们设计并出售期货合约，例如投行，掉期交易商或其它衍生品公司
* Asset Manager/Institutional: 资产管理人/机构，机构投资者，例如养老基金，捐赠基金，保险公司和共同基金等
* Leveraged Funds: 杠杆基金，主要指对冲基金，CTA, CPO等机构
* Other Reportables: 不属于上述三个类别的其它参与者，一般使用期货对冲风险

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## 数据

- 从[官网](https://publicreporting.cftc.gov/stories/s/r4w3-av2u)下载COT报告，存储到本地csv
- 从雅虎财经下载关键资产的历史价格数据，存储到本地csv

**关于COT报告**

下载的持仓报告包含所有产品的历史持仓，根据合约产品代码来筛选特定合约，例如：

- ICE美元指数：098662
- 迷你标普500指数：13874A
- 黄金：088691

产品代码可以在[这里](https://www.tradingster.com/)找到。

In [11]:
# COT报告文件路径
cot_file_path = "~/Downloads/cot_legacy_futures_only.csv"

# CFTC合约市场代码
code = "098662"

# 价格数据文件路径
price_file_path = "../data/yahoo/ICE US Dollar Index.csv"

# 读取COT报告数据
cot = pd.read_csv(cot_file_path)

# 读取价格数据
price = pd.read_csv(price_file_path, index_col=0, parse_dates=True)

清洗和合并数据集

In [3]:
# COT报告包含很多字段，我们只需要最关键的几个字段
fields = {
    "Report_Date_as_YYYY_MM_DD": "date",
    "CFTC_Contract_Market_Code": "code",
    "Commodity Name": "commodity",
    "NonComm_Positions_Long_All": "non_comm_long",
    "NonComm_Positions_Short_All": "non_comm_short",
}

# 清洗期货持仓数据
clean_cot = (
    cot.query("CFTC_Contract_Market_Code == @code")
    .loc[:, list(fields.keys())]
    .rename(columns=fields)
    .assign(date=lambda x: pd.to_datetime(x["date"], format="%m/%d/%Y %I:%M:%S %p"))
    .set_index("date")
    .sort_index()
)

# 合并数据集
df = (
    clean_cot.join(price["Close"], how="left")
    .rename(columns={"Close": "price"})
    .dropna()
)

In [4]:
df

,code,commodity,non_comm_long,non_comm_short,price
date,,,,,
2000-01-04,098662,U.S. DOLLAR INDEX,2228,1967,100.410004
2000-01-11,098662,U.S. DOLLAR INDEX,3666,6236,100.559998
2000-01-18,098662,U.S. DOLLAR INDEX,2934,1677,101.860001
2000-01-25,098662,U.S. DOLLAR INDEX,2800,1524,102.410004
2000-02-01,098662,U.S. DOLLAR INDEX,2605,1791,104.919998
...,...,...,...,...,...
2024-12-31,098662,U.S. DOLLAR INDEX,26598,19739,108.489998
2025-01-07,098662,U.S. DOLLAR INDEX,27958,19328,108.540001
2025-01-14,098662,U.S. DOLLAR INDEX,29512,16817,109.269997


## 情绪指标

期货持仓通常用于衡量市场情绪，主要有两种方法：

1. 计算非商业期货净头寸，然后进行标准化，例如计算52周滚动标准分数。
2. 计算非商业期货多头持仓占总持仓的比例。

以方法1为例，当非商业期货净头寸的滚动标准分数高于2，说明投机者全面做多，市场情绪可能过度乐观，中长期价格可能筑顶并开始下跌；当标准分数低于-2，说明投机者全面做空，市场情绪可能过度悲观，中长期价格可能见底并开始上涨。

从逆向交易的逻辑来解读持仓数据。

### 1. 滚动标准分数

In [5]:
# 计算标准分数的滚动窗口，52周通常已经足够
zscore_window = 52

# 计算非商业期货净头寸及其标准分数
df["non_comm_net"] = df["non_comm_long"] - df["non_comm_short"]
df["non_comm_net_zscore"] = (
    df["non_comm_net"] - df["non_comm_net"].rolling(zscore_window).mean()
) / df["non_comm_net"].rolling(zscore_window).std()

df

,code,commodity,non_comm_long,non_comm_short,price,non_comm_net,non_comm_net_zscore
date,,,,,,,
2000-01-04,098662,U.S. DOLLAR INDEX,2228,1967,100.410004,261,NaN
2000-01-11,098662,U.S. DOLLAR INDEX,3666,6236,100.559998,-2570,NaN
2000-01-18,098662,U.S. DOLLAR INDEX,2934,1677,101.860001,1257,NaN
2000-01-25,098662,U.S. DOLLAR INDEX,2800,1524,102.410004,1276,NaN
2000-02-01,098662,U.S. DOLLAR INDEX,2605,1791,104.919998,814,NaN
...,...,...,...,...,...,...,...
2024-12-31,098662,U.S. DOLLAR INDEX,26598,19739,108.489998,6859,0.195115
2025-01-07,098662,U.S. DOLLAR INDEX,27958,19328,108.540001,8630,0.410550
2025-01-14,098662,U.S. DOLLAR INDEX,29512,16817,109.269997,12695,0.904942


查看非商业期货多头，空头和净头寸的变化。

In [6]:
fig = px.line(
    df,
    x=df.index,
    y=["non_comm_long", "non_comm_short", "non_comm_net"],
    title="Non-Commercial Positions",
)

fig.for_each_trace(lambda t: t.update(name=t.name.capitalize().replace("_", " ")))

fig.update_layout(
    width=1000,
    height=600,
    xaxis=dict(title="Date"),
    yaxis=dict(title="# of Contracts"),
    legend=dict(
        orientation="h", xanchor="center", yanchor="bottom", x=0.5, y=1.02, title=""
    ),
)

fig.show()

查看滚动标准分数，评估市场情绪

In [7]:
fig = go.Figure()

# 非商业期货净头寸的标准分数
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df["non_comm_net_zscore"],
        name="Zscore of Non-Commercial Net Positions",
        yaxis="y1",
        line=dict(color="blue"),
    )
)

# 合约价格
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df["price"],
        name="US Dollar Index",
        yaxis="y2",
        line=dict(color="green"),
    )
)

# 添加水平线代表极端情绪
levels = [-3, -2, -1, 1, 2, 3]
for level in levels:
    fig.add_hline(y=level, line=dict(color="red", width=1, dash="dot"))

fig.update_layout(
    title="Zscore of Non-Commercial Net Positions",
    width=1000,
    height=600,
    xaxis=dict(title="Date"),
    yaxis=dict(title="Zscore"),
    yaxis2=dict(title="US Dollar Index", overlaying="y", side="right"),
    legend=dict(orientation="h", xanchor="center", yanchor="bottom", x=0.5, y=1.02),
)

fig.show()

### 2. 多头持仓比例

In [8]:
from indicators import lowpass_filter

In [9]:
df["long_ratio"] = df["non_comm_long"] / (df["non_comm_long"] + df["non_comm_short"])
df["long_ratio_smooth"] = lowpass_filter(df["long_ratio"], 8)
# df

In [10]:
fig = go.Figure()

# 多头比例
# fig.add_trace(
#     go.Scatter(
#         x=df.index,
#         y=df["long_ratio"],
#         name="Long Ratio",
#         yaxis="y1",
#         line=dict(color="blue"),
#     )
# )

# 平滑后的多头比例
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df["long_ratio_smooth"],
        name="Smoothed Long Ratio",
        yaxis="y1",
        line=dict(color="blue"),
    )
)

# 合约价格
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df["price"],
        name="US Dollar Index",
        yaxis="y2",
        line=dict(color="green"),
    )
)

# 添加水平线代表极端情绪
levels = [0.2, 0.7, 0.8, 0.9]
for level in levels:
    fig.add_hline(y=level, line=dict(color="red", width=1, dash="dot"))

fig.update_layout(
    title="Non-Commercial Long Ratio",
    width=1000,
    height=600,
    xaxis=dict(title="Date"),
    yaxis=dict(title="Ratio"),
    yaxis2=dict(title="US Dollar Index", overlaying="y", side="right"),
    legend=dict(orientation="h", xanchor="center", yanchor="bottom", x=0.5, y=1.02),
)

fig.show()

In [14]:
cot_file_path = "~/Downloads/cot_ttf_futures_only.csv"

cot = pd.read_csv(cot_file_path)

# cot.head()
cot.columns

Index(['ID', 'Market_and_Exchange_Names', 'Report_Date_as_YYYY_MM_DD',
       'YYYY Report Week WW', 'CONTRACT_MARKET_NAME',
       'CFTC_Contract_Market_Code', 'CFTC_Market_Code', 'CFTC_Region_Code',
       'CFTC_Commodity_Code', 'Commodity Name', 'Open_Interest_All',
       'Dealer_Positions_Long_All', 'Dealer_Positions_Short_All',
       'Dealer_Positions_Spread_All', 'Asset_Mgr_Positions_Long_All',
       'Asset_Mgr_Positions_Short_All', 'Asset_Mgr_Positions_Spread_All',
       'Lev_Money_Positions_Long_All', 'Lev_Money_Positions_Short_All',
       'Lev_Money_Positions_Spread_All', 'Other_Rept_Positions_Long_All',
       'Other_Rept_Positions_Short_All', 'Other_Rept_Positions_Spread_All',
       'Tot_Rept_Positions_Long_All', 'Tot_Rept_Positions_Short_All',
       'NonRept_Positions_Long_All', 'NonRept_Positions_Short_All',
       'Change_in_Open_Interest_All', 'Change_in_Dealer_Long_All',
       'Change_in_Dealer_Short_All', 'Change_in_Dealer_Spread_All',
       'Change_in_Asset_M

In [15]:
code = "098662"

fields = {
    "Report_Date_as_YYYY_MM_DD": "date",
    "CFTC_Contract_Market_Code": "code",
    "Commodity Name": "commodity",
    "Dealer_Positions_Long_All": "dealer_long",
    "Dealer_Positions_Short_All": "dealer_short",
    "Asset_Mgr_Positions_Long_All": "asset_mgr_long",
    "Asset_Mgr_Positions_Short_All": "asset_mgr_short",
    "Lev_Money_Positions_Long_All": "lev_money_long",
    "Lev_Money_Positions_Short_All": "lev_money_short",
    "Other_Rept_Positions_Long_All": "other_rept_long",
    "Other_Rept_Positions_Short_All": "other_rept_short",
    "NonRept_Positions_Long_All": "non_rept_long",
    "NonRept_Positions_Short_All": "non_rept_short",
}

clean_cot = (
    cot.query("CFTC_Contract_Market_Code == @code")
    .loc[:, list(fields.keys())]
    .rename(columns=fields)
    .assign(date=lambda x: pd.to_datetime(x["date"], format="%m/%d/%Y %I:%M:%S %p"))
    .set_index("date")
    .sort_index()
)

In [17]:
# clean_cot

In [18]:
fig = px.line(
    clean_cot,
    x=clean_cot.index,
    y=[
        "dealer_long",
        "asset_mgr_long",
        "lev_money_long",
        "other_rept_long",
        "non_rept_long",
    ],
    title="Long Positions",
)

fig.update_layout(
    width=1000,
    height=600,
    xaxis=dict(title="Date"),
    yaxis=dict(title="# of Contracts"),
    legend=dict(
        orientation="h", xanchor="center", yanchor="bottom", x=0.5, y=1.02, title=""
    ),
)

fig.show()

In [19]:
fig = px.line(
    clean_cot,
    x=clean_cot.index,
    y=[
        "dealer_short",
        "asset_mgr_short",
        "lev_money_short",
        "other_rept_short",
        "non_rept_short",
    ],
    title="Long Positions",
)

fig.update_layout(
    width=1000,
    height=600,
    xaxis=dict(title="Date"),
    yaxis=dict(title="# of Contracts"),
    legend=dict(
        orientation="h", xanchor="center", yanchor="bottom", x=0.5, y=1.02, title=""
    ),
)

fig.show()

In [20]:
df = clean_cot.copy()

df["dealer_long_ratio"] = df["dealer_long"] / (df["dealer_long"] + df["dealer_short"])
df["asset_mgr_long_ratio"] = df["asset_mgr_long"] / (
    df["asset_mgr_long"] + df["asset_mgr_short"]
)
df["lev_money_long_ratio"] = df["lev_money_long"] / (
    df["lev_money_long"] + df["lev_money_short"]
)
df["other_rept_long_ratio"] = df["other_rept_long"] / (
    df["other_rept_long"] + df["other_rept_short"]
)
df["non_rept_long_ratio"] = df["non_rept_long"] / (
    df["non_rept_long"] + df["non_rept_short"]
)

In [34]:
columns = [
    "dealer_long_ratio",
    "asset_mgr_long_ratio",
    "lev_money_long_ratio",
    "other_rept_long_ratio",
    "non_rept_long_ratio",
]

fig = make_subplots(
    rows=5, cols=1, shared_xaxes=True, subplot_titles=columns, vertical_spacing=0.04
)

for i, column in enumerate(columns, 1):
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df[column],
            name=column.replace("_", " ").capitalize(),
        ),
        row=i,
        col=1,
    )

fig.update_layout(width=1000, height=800, title="Long Ratio", showlegend=False)

多头比例似乎是捕捉趋势的有效指标，当多头比例高于0.7，市场维持强势，这种解读是否跟逆向思维冲突？

可能不冲突，一个指标可以有多种不同的解读方式，有时候可以识别趋势，有时候可以识别周期。

优化研究报告的内容：
- 分别研究传统报告和金融交易者报告，前者的作用似乎更高，后者包含更多细分种类，但是如何解读尚不明确
- 深入研究多头比例，分别用于识别趋势和周期